In [1]:
pip install adversarial-robustness-toolbox


     |████████████████████████████████| 655kB 2.8MB/s 
     |████████████████████████████████| 235kB 13.8MB/s 
     |████████████████████████████████| 20.8MB 1.3MB/s 
     |████████████████████████████████| 6.8MB 56.3MB/s 
     |████████████████████████████████| 747kB 54.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [20]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch
import torchvision.models as models
from torchvision import transforms as T

from PIL import Image
from art.attacks.evasion import *
from art.estimators.classification import PyTorchClassifier

transform = T.Compose([            
 T.Resize(256),                    
 T.CenterCrop(224),                
 T.ToTensor(),                     
 T.Normalize(mean=[0.485, 0.456, 0.406],
             std=[0.229, 0.224, 0.225])
 ])

In [24]:
import urllib
url, filename = ("https://www.insidehook.com/wp-content/uploads/2020/04/JetBlue-Airplane.jpg?fit=1500%2C1000", "example.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

img = Image.open('airplane.jpg')
transformed_img = transform(img)
input_img = transformed_img.unsqueeze(0)

In [25]:
arr = []
image_array = np.asarray(img)

x = np.moveaxis(image_array, -1, 0)
arr.append(x)
arr = np.array(arr, dtype = 'float32')
z =  np.moveaxis(x, 0, -1)


In [26]:
model = models.resnet18(pretrained=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 224, 224),
    nb_classes=1000,
)

In [27]:
e = 0.01
attack = DeepFool(classifier, epsilon=e)
advx = attack.generate(x=arr)



DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]

DeepFool: 100%|██████████| 1/1 [01:02<00:00, 62.32s/it]


In [28]:
rescaled = (255*np.transpose(advx[0])).astype(np.uint8)
inp_img = Image.fromarray(rescaled)
s = 'DeepFoolPlane' + str(e) + '.png'
inp_img.save(s, 'png')